In [3]:
import pandas as pd
data = pd.read_csv("Hall_votes.csv")

In [4]:
import numpy as np
rec =[]
for i in range(21104606):
    if (data["congress"][i] == 117):
        rec.append(np.array([data["rollnumber"][i],data["icpsr"][i],data["cast_code"][i]]))
del data

In [5]:
rec = np.array(rec)

In [6]:
lrec = [rec[rec[:, 0] == val] for val in (range(1,997))]
del rec

In [7]:
common_values = np.unique(lrec[0][:,1])
for i in range(996):
    common_values = np.intersect1d(np.unique(lrec[i][:,1]),common_values)

In [8]:
common_values = common_values[:-1]

In [9]:
common_values.shape

(411,)

In [10]:
votes = np.zeros((996,411))
for i in range(996):
    for j in range(411):
        a = np.where(lrec[i][:,1] == common_values[j])
        votes[i,j] = lrec[i][a,2]

In [11]:
np.unique(votes)

array([1., 6., 7., 9.])

In [12]:
for i in range(996):
    for j in range(411):
        if votes[i,j] == 7 or votes[i,j] == 9:
            votes[i,j] = np.random.binomial(1, 0.5)
        if (votes[i,j] == 6):
            votes[i,j] = 0

In [13]:
np.unique(votes)

array([0., 1.])

In [14]:
votes.shape

(996, 411)

In [15]:
result = np.zeros(996)
for i in range(996):
    if (np.sum(votes[i,:])<=211):
        result[i] = 0
    else:
        result[i] = 1

In [16]:
import gurobipy as gp
from gurobipy import GRB

In [76]:
n = 996
m = 411
model = gp.Model('voting')
rep = model.addVars(m,lb=-1.0, ub=1.0, vtype=GRB.CONTINUOUS, name="rep")
bill = model.addVars(n,lb=-1.0, ub=1.0, vtype=GRB.CONTINUOUS, name="bill")
Z = model.addVars(n, vtype=GRB.BINARY, name="Z")
sign = model.addVars(n,m, vtype=GRB.BINARY, name="sign")
sign2 = model.addVars(n, vtype=GRB.BINARY, name="sign2")
sign3 = model.addVars(n,n, vtype=GRB.BINARY, name="sign3")

In [80]:
M = 2.01
M2 = 413
sign3 = np.zeros((n,n))
for i in range(n):
    for j in range(m):
        model.addConstr((rep[j]-bill[i]) <= (M*sign[i,j]-0.01))
        model.addConstr((bill[i]-rep[j]) <= (M*(1-sign[i,j])-0.01))
    model.addConstr((sum(sign[i,j] for j in range(m)) - 210.5 )<=M2*sign2[i])
    model.addConstr((-sum(sign[i,j] for j in range(m)) +210.5)<=M2*(1-sign2[i]))
    model.addConstr(Z[i]>=sign2[i]-result[i])
    model.addConstr(Z[i]>=-sign2[i]+result[i])
for i in range(0,n-3):
    for j in range(i,20):
        model.addConstr((bill[i]-bill[i+n-j-1]) <= (M*sign3[i,j]-0.001))
        model.addConstr((bill[i+n-j-1]-bill[i]) <= (M*(1-sign3[i,j])-0.001))

In [ ]:
model.setObjective(sum(Z[i] for i in range(n)),GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1645812 rows, 1404771 columns and 6562488 nonzeros
Model fingerprint: 0xcd1986b4
Variable types: 1407 continuous, 1403364 integer (1403364 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-05, 2e+02]

MIP start from previous solve did not produce a new incumbent solution
Processed MIP start in 2.10 seconds (1.58 work units)

Found heuristic solution: objective 754.0000000
Presolve removed 824898 rows and 993012 columns (presolve time = 5s) ...
Presolve removed 1610994 rows and 1387014 columns (presolve time = 10s) ...
Presolve removed 1610994 rows and 1387014 columns
Presolve time: 13.81s
Presolved: 34818 rows, 17757 columns, 138600 nonzeros
V

  5903  2833    7.00000  329   93    8.00000    0.00000   100%   396 1355s
  6121  2873    7.00000  342   93    8.00000    0.00000   100%   396 1362s
  6325  2882    7.00000  354   92    8.00000    0.00000   100%   394 1371s
  6488  2882    7.00000  367   92    8.00000    0.00000   100%   396 1377s
  6663  2902    7.00000  389  122    8.00000    0.00000   100%   399 1384s
  6855  2912    7.00000  419   95    8.00000    0.00000   100%   404 1396s
  7049  3003    7.00000  445   95    8.00000    0.00000   100%   413 1408s
  7268  3182    7.00000  466   95    8.00000    0.00000   100%   420 1419s
  7605  3352    7.00000  490   95    8.00000    0.00000   100%   415 1431s
  7929  3399    7.00000  513   95    8.00000    0.00000   100%   409 1553s
  8049  3465    1.00000   25 7429    8.00000    0.00000   100%   411 1609s
  8243  3470    5.00000   30 11696    8.00000    0.00000   100%   409 1703s
  8254  3757    5.00000   31 6075    8.00000    0.00000   100%   410 1711s
  8710  3909    6.00000 

 114233 62101    7.00000 1013  551    8.00000    0.00000   100%   235 6380s
 115322 62189 infeasible 1085         8.00000    0.00000   100%   235 6419s
 115614 62709    0.00000 1119  267    8.00000    0.00000   100%   235 6433s
 116657 63198    1.00000 1178  320    8.00000    0.00000   100%   234 6448s
 117661 63937    6.00000 1251  367    8.00000    0.00000   100%   235 6463s
 119064 65048 infeasible 1308         8.00000    0.00000   100%   234 6480s
 120951 66650    7.00000 1200  357    8.00000    0.00000   100%   233 6525s
 123485 66897    0.00000  916  220    8.00000    0.00000   100%   230 6628s
 123822 68376    0.00000  920  221    8.00000    0.00000   100%   230 6650s
 126399 69608    6.00000 1083  345    8.00000    0.00000   100%   228 6665s
 128113 70796    7.00000 1009  275    8.00000    0.00000   100%   226 6680s
 129867 71405 infeasible 1171         8.00000    0.00000   100%   225 6693s
 130986 71985    7.00000 1083  500    8.00000    0.00000   100%   224 6706s
 132089 7287

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "D:\Anaconda 3\Anaconda\lib\site-packages\ipykernel\iostream.py", line 502, in write
    def write(self, string: str) -> int:
KeyboardInterrupt: 


 186450 103802    2.00000  700  301    8.00000    0.00000   100%   207 15077s
 187882 104110 infeasible  800         8.00000    0.00000   100%   206 15092s
 188731 104563    7.00000  906  284    8.00000    0.00000   100%   206 15105s
 189641 104619    0.00000  776  305    8.00000    0.00000   100%   206 15264s
 189792 104622    0.00000  774  285    8.00000    0.00000   100%   206 15318s
 189809 104628 infeasible  770         8.00000    0.00000   100%   206 15361s
 189829 105060    0.00000  777  313    8.00000    0.00000   100%   206 15376s
 190868 105474 infeasible  788         8.00000    0.00000   100%   206 15394s
 191926 106086 infeasible  825         8.00000    0.00000   100%   207 15411s
 193248 106931    1.00000  765  365    8.00000    0.00000   100%   206 15427s
 194722 107770 infeasible  804         8.00000    0.00000   100%   206 15446s
 196243 108312 infeasible  710         8.00000    0.00000   100%   205 15545s
 197358 108637    7.00000  741  191    8.00000    0.00000   100%

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "D:\Anaconda 3\Anaconda\lib\site-packages\ipykernel\iostream.py", line 502, in write
    def write(self, string: str) -> int:
KeyboardInterrupt: 


 208693 115635    0.00000 1038  311    8.00000    0.00000   100%   202 15910s
 210370 116561    0.00000  734  230    8.00000    0.00000   100%   202 15930s
 212045 118098    5.00000 1053  667    8.00000    0.00000   100%   201 15965s
 214785 118408    2.00000  665  604    8.00000    0.00000   100%   200 16249s
 215407 118417    3.00000  669  417    8.00000    0.00000   100%   200 16328s
 215431 118423    4.00000  670  797    8.00000    0.00000   100%   200 16444s
 215441 118503 infeasible  671         8.00000    0.00000   100%   200 16490s
 215557 119310    2.00000  671  927    8.00000    0.00000   100%   200 16520s
 217419 120008    7.00000  718  351    8.00000    0.00000   100%   200 16544s
 218641 120008    0.00000  636  209    8.00000    0.00000   100%   200 16545s
 218905 120743    0.00000  820  345    8.00000    0.00000   100%   200 16567s
 220333 121703    6.00000  857  314    8.00000    0.00000   100%   200 16590s
 222038 122688 infeasible  786         8.00000    0.00000   100%

In [70]:
print(result)

[0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1.
 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0.
 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1.
 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.